Conduct Multiple Experiments ChipWhisperer into Custom File Format

In [ ]:
from WPI_SCA_LIBRARY.CWScope import *
from WPI_SCA_LIBRARY.LeakageModels import *

# initialize scope
scope = CWScope("../WPI_SCA_LIBRARY/firmware/simpleserial-aes-CWLITEARM.hex")
 
# capture 1000 traces and store in an experiment named CWCapture1
scope.cw_to_file_format(1000, file_name="TestFile", experiment_name="CWCapture1", file_existing=False)

# capture 1500 traces and store in an experiment named CWCapture2
scope.cw_to_file_format(1500, file_name="TestFile", experiment_name="CWCapture2", file_existing=True)

# capture two traces sets for t-test and put into one experiment
fixed_t, rand_t = scope.capture_traces_tvla(num_traces=500)

# add t-test experiment to TestFile
file_parent = FileFormatParent("TestFile", existing=True)
file_parent.addExperiment(name="T_test", path="T_test", existing=False)

# add both datasets to file
exp = file_parent.getExperiment("T_test")

exp.createDataset("FixedTraces", "fixed", size=(500, scope.scope.adc.samples), type='uint8')
fixed_data = exp.getDataset("FixedTraces")
fixed_data.addData(index=range(500), dataToAdd=fixed_t)

exp.createDataset("RandomTraces", "random", size=(500, scope.scope.adc.samples), type='uint8')
random_data = exp.getDataset("RandomTraces")
random_data.addData(index=range(500), dataToAdd=rand_t)

__SNR__

In [ ]:
from WPI_SCA_LIBRARY.CWScope import *
from WPI_SCA_LIBRARY.LeakageModels import *
# read in data from CWCapture2 experiment
file = FileFormatParent("TestFile", existing=True)
experiment = file.getExperiment("CWCapture2")

print(experiment.dataset)

traces = experiment.getDataset("CWCapture2Traces").readAll()
keys = experiment.getDataset("CWCapture2Keys").readAll()
plaintexts = experiment.getDataset("CWCapture2Plaintexts").readAll()

# create labels for SNR
labels = Sbox[keys ^ plaintexts] 
labels = labels[:, 1]
labels = np.reshape(labels, (len(keys), 1))

# create a dataset in the CWCapture2 experiment to hold the labels 
experiment.createDataset("labelsB", "labelsB", size = (len(keys), 1), type = 'int8')
experiment.getDataset("labelsB").addData(range(len(keys)),labels)

# use integrated metric to run SNR 
test = experiment.calculateSNR("labelsB", "CWCapture2Traces", visualise=True, saveGraph=True)

General File Format Demo